# Création du dataset final

## Import des bibiliothèques

In [64]:
import pandas as pd
import os
import glob
from datetime import datetime
from pathlib import Path

## Récupération des données brutes dans le répertoire /data/raw

In [65]:
repertoire_data_raw = Path.cwd().parent.parent / "data" / "raw"

fichiers_csv = [f for f in list(repertoire_data_raw.glob("*.csv")) if "metadata" not in f.name.lower() and "~" not in f.name]

for file_path in fichiers_csv:
    print(f"Lecture du fichier CSV : {file_path.name}")

fichiers_excel = [f for f in list(repertoire_data_raw.glob("*.xlsx")) if "metadata" not in f.name.lower() and "~" not in f.name]

for file_path in fichiers_excel:
    print(f"Lecture du fichier Excel : {file_path.name}")

Lecture du fichier CSV : défaillances_entreprises-tous_secteurs-brut-france-1990_2025.csv
Lecture du fichier CSV : euribor-3_month-1994_2025.csv
Lecture du fichier CSV : indicateur_climat_affaires-tous_secteurs-france_métropolitaine-1977_2025.csv
Lecture du fichier CSV : indicateur_climat_emploi-tous_secteurs-france_métropolitaine-1991_2025.csv
Lecture du fichier CSV : indicateur_retournement_conjoncturel-tous_secteurs-france_métropolitaine-1988_2025.csv
Lecture du fichier CSV : indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025.csv
Lecture du fichier CSV : ipc-glissement_annuel-ensemble_menage-1991_2025.csv
Lecture du fichier CSV : ipc_energie_only-glissement_annuel-ensemble_menage-1991_2025.csv
Lecture du fichier CSV : isj-glissement_annuel-ensemble_menage-1991_2025.csv
Lecture du fichier CSV : main_refinancing_operations-1999_2025.csv
Lecture du fichier CSV : ocde-taux_chomage_mensuel-1984_2025.csv
Lecture du fichier CSV : pib_total-volume_prix_année_préc

In [66]:
# Dictionnaire pour stocker les dataframes
dict_dfs = {}

# Importation des CSV
for file_path in fichiers_csv:
    # On utilise le nom du fichier (sans .csv) comme clé
    nom_cle = file_path.stem 
    print(f"Importation de : {nom_cle}")
    # Lecture personnalisée (ajustez sep et encoding selon vos tests précédents)
    if nom_cle == 'euribor-3_month-1994_2025' or nom_cle == 'main_refinancing_operations-1999_2025' or nom_cle == 'taux_chomage-ocde-mensuel-1984_2025':
        df = pd.read_csv(file_path, sep=',', encoding='latin1')
    else:
        df = pd.read_csv(file_path, sep=';', encoding='utf-8')
    # Stockage dans le dictionnaire
    dict_dfs[nom_cle] = df

# Importation des EXCEL
for file_path in fichiers_excel:
    nom_cle = file_path.stem 
    print(f"Importation de : {nom_cle}")
    # Lecture personnalisée (ajustez sep et encoding selon vos tests précédents)
    if nom_cle == 'séries_CVS_T22025':
        df = pd.read_excel(file_path, sheet_name='DEFM cat ABC_age_sexe',header=6)
    elif nom_cle == 'interimaire-evolution-trimestrielle-2000_2025':
        df = pd.read_excel(file_path, sheet_name="Nombre d'intérimaires",header=7)
    elif nom_cle == 'taux_chomage-insee-annuel-1975_2024':
        df = pd.read_excel(file_path, sheet_name="Figure 5",header=2)
    elif nom_cle == 'taux_chomage_sexe_age-insee-trimestriel-1975_2025':
        df = pd.read_excel(file_path, sheet_name="France - Figure 2",header=3)
        
    else:
        df = pd.read_excel(file_path, sheet_name=0)

    # Stockage dans le dictionnaire
    dict_dfs[nom_cle] = df


    
# Pour voir la liste des fichiers chargés :
#print("\nFichiers chargés :", dict_dfs.keys())

Importation de : défaillances_entreprises-tous_secteurs-brut-france-1990_2025
Importation de : euribor-3_month-1994_2025
Importation de : indicateur_climat_affaires-tous_secteurs-france_métropolitaine-1977_2025
Importation de : indicateur_climat_emploi-tous_secteurs-france_métropolitaine-1991_2025
Importation de : indicateur_retournement_conjoncturel-tous_secteurs-france_métropolitaine-1988_2025
Importation de : indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025
Importation de : ipc-glissement_annuel-ensemble_menage-1991_2025
Importation de : ipc_energie_only-glissement_annuel-ensemble_menage-1991_2025
Importation de : isj-glissement_annuel-ensemble_menage-1991_2025
Importation de : main_refinancing_operations-1999_2025
Importation de : ocde-taux_chomage_mensuel-1984_2025
Importation de : pib_total-volume_prix_année_précédente_chaînés-série_CVS-CJO-1949_2025
Importation de : taux_chomage-ocde-mensuel-1984_2025
Importation de : interimaire-evolution-trimestri

## Création du dataframe template dans lequel on ajoutera les données des fichiers brutes

In [67]:
# 1. Définir le début et la fin (aujourd'hui)
date_debut = "1990-01-01"
date_fin = datetime.now()

# 2. Créer la plage de dates mensuelle
# 'MS' signifie 'Month Start' (1er du mois)
# 'ME' signifierait 'Month End' (dernier jour du mois)
date_periode = pd.date_range(start=date_debut, end=date_fin, freq='MS')

# 3. Créer le DataFrame
df = pd.DataFrame({'date': date_periode})

# 4. Extraire les colonnes demandées précédemment (Année, Mois, Trimestre)
df['annee'] = df['date'].dt.year
df['trimestre'] = df['date'].dt.quarter
df['mois'] = df['date'].dt.month

# Traitement des données brutes

## 1) Taux de chômage en france par sexe et âge (INSEE)

In [68]:
# taux_chomage_sexe_age-insee-trimestriel-1975_2025
df_tx_chomage_sexe_age = dict_dfs['taux_chomage_sexe_age-insee-trimestriel-1975_2025']

# Renommer les colonnes pour les uniformiser
df_tx_chomage_sexe_age = df_tx_chomage_sexe_age.rename(columns={
    'Femmes': 'taux_chomage_femme_insee',
    'Hommes': 'taux_chomage_homme_insee',
    '15-24 ans': 'taux_chomage_15_24_insee',
    '25-49 ans': 'taux_chomage_25_49_insee',
    '50 ans ou plus': 'taux_chomage_50_plus_insee',
    'Ensemble': 'taux_chomage_total_insee'
})

df_tx_chomage_sexe_age['Période'] = df_tx_chomage_sexe_age['Année'].astype(str) + df_tx_chomage_sexe_age['Trimestre']

# Modification de la colonne de temps
df_tx_chomage_sexe_age['time_period'] = pd.to_datetime(df_tx_chomage_sexe_age['Période'].str.replace('T', '-Q'))
#df_defm_abcd['time_period'] = pd.to_datetime(df_defm_abcd['time_period'], dayfirst=True)
df_tx_chomage_sexe_age['date_jointure'] = df_tx_chomage_sexe_age['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df, 
        df_tx_chomage_sexe_age[['date_jointure', 'taux_chomage_total_insee', 'taux_chomage_homme_insee', 'taux_chomage_femme_insee',
                       'taux_chomage_15_24_insee', 'taux_chomage_25_49_insee', 'taux_chomage_50_plus_insee'
                      ]], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_22484\1696968869.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_tx_chomage_sexe_age['time_period'] = pd.to_datetime(df_tx_chomage_sexe_age['Période'].str.replace('T', '-Q'))


In [69]:
df_final

,date,annee,trimestre,mois,taux_chomage_total_insee,taux_chomage_homme_insee,taux_chomage_femme_insee,taux_chomage_15_24_insee,taux_chomage_25_49_insee,taux_chomage_50_plus_insee
0,1990-01-01,1990,1,1,8.1,6.3,10.3,16.9,6.9,5.1
1,1990-02-01,1990,1,2,NaN,NaN,NaN,NaN,NaN,NaN
2,1990-03-01,1990,1,3,NaN,NaN,NaN,NaN,NaN,NaN
3,1990-04-01,1990,2,4,8.0,6.3,10.2,16.8,6.8,5.1
4,1990-05-01,1990,2,5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
429,2025-10-01,2025,4,10,7.9,8.1,7.6,21.5,6.9,5.1
430,2025-11-01,2025,4,11,NaN,NaN,NaN,NaN,NaN,NaN
431,2025-12-01,2025,4,12,NaN,NaN,NaN,NaN,NaN,NaN
432,2026-01-01,2026,1,1,NaN,NaN,NaN,NaN,NaN,NaN


## 1) Demandeurs d'emploi inscrits à France Travail, en catégories A, B, C, D (DEFM ABCD)

In [70]:
# Demandeurs d'emploi inscrits à France Travail en catégories A, B, C, D - DEFM ABCD
df_defm_abcd = dict_dfs['séries_CVS_T22025']
df_defm_abcd

,Unnamed: 0,Moins de 25 ans,25 à 49 ans,50 ans ou plus,Ensemble,Moins de 25 ans.1,25 à 49 ans.1,50 ans ou plus .1,Ensemble.1,Moins de 25 ans.2,...,DEFM B,DEFM C,DEFM ABC,DEFM A.1,DEFM B.1,DEFM C.1,DEFM ABC.1,Part des hommes en activité réduite parmi la catégorie ABC,Part des femmes en activité réduite parmi la catégorie ABC,Part de l'activité réduite dans la catégorie ABC
0,1996T1,387200,1222900,224600,1834700,462200,1463300,201000,2126500,849400,...,108600,162800,1834700,1761600,180800,184100,2126500,0.147926,0.171597,0.160633
1,1996T2,389700,1236100,228900,1854700,463100,1473500,205800,2142400,852800,...,114300,172200,1854700,1766400,185600,190400,2142400,0.154472,0.175504,0.165745
2,1996T3,396000,1257400,234200,1887500,467700,1490500,211600,2169800,863700,...,120300,194400,1887500,1770500,194400,204900,2169800,0.166728,0.184026,0.175979
3,1996T4,400500,1278900,240000,1919400,464900,1502100,217100,2184100,865400,...,126600,193200,1919300,1772500,203200,208400,2184100,0.166623,0.188453,0.178242
4,1997T1,402000,1298700,246800,1947500,465100,1519000,223200,2207300,867100,...,128300,207200,1947400,1779700,207800,219800,2207300,0.172281,0.193721,0.183672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,2024T2,356600,1561400,705000,2623000,332100,1659700,775000,2766800,688700,...,382800,707700,2623000,1481600,466300,818900,2766800,0.415745,0.464508,0.440777
114,2024T3,359600,1568500,709600,2637700,333100,1664000,777600,2774700,692700,...,384400,708800,2637800,1488300,471200,815200,2774700,0.414436,0.463618,0.439649
115,2024T4,371700,1591600,715600,2678900,345100,1687400,780300,2812800,716700,...,375200,707200,2679000,1538500,458500,815800,2812800,0.404031,0.453036,0.429131
116,2025T1,453800,1631700,725500,2811000,413000,1725900,788100,2927100,866800,...,365800,705200,2810900,1668100,447500,811500,2927100,0.381017,0.430119,0.406065


In [71]:
# Demandeurs d'emploi inscrits à France Travail en catégories A, B, C, D - DEFM ABCD
df_defm_abcd = dict_dfs['séries_CVS_T22025']

# Renommer les colonnes pour les uniformiser
df_defm_abcd = df_defm_abcd.rename(columns={
    'Unnamed: 0': 'Période',
    'Moins de 25 ans': 'demandeur_homme_abcd_moins25',
    '25 à 49 ans': 'demandeur_homme_abcd_2549',
    '50 ans ou plus ': 'demandeur_homme_abcd_plus50',
    'Moins de 25 ans.1': 'demandeur_femme_abcd_moins25',
    '25 à 49 ans.1': 'demandeur_femme_abcd_2549',
    '50 ans ou plus .1': 'demandeur_femme_abcd_plus50'
})

# Modification de la colonne de temps
df_defm_abcd['time_period'] = pd.to_datetime(df_defm_abcd['Période'].str.replace('T', '-Q'))
#df_defm_abcd['time_period'] = pd.to_datetime(df_defm_abcd['time_period'], dayfirst=True)
df_defm_abcd['date_jointure'] = df_defm_abcd['time_period'].dt.to_period('M').dt.to_timestamp()

# Création des colonnes "TOTAL AGE"
df_defm_abcd['demandeur_total_abcd_moins25'] = df_defm_abcd['demandeur_homme_abcd_moins25'] + df_defm_abcd['demandeur_femme_abcd_moins25']
df_defm_abcd['demandeur_total_abcd_2549']    = df_defm_abcd['demandeur_homme_abcd_2549']    + df_defm_abcd['demandeur_femme_abcd_2549']
df_defm_abcd['demandeur_total_abcd_plus50']  = df_defm_abcd['demandeur_homme_abcd_plus50']  + df_defm_abcd['demandeur_femme_abcd_plus50']

# Création des colonnes "TOTAL SEXE"
df_defm_abcd['demandeur_homme_abcd_total']   = df_defm_abcd['demandeur_homme_abcd_moins25'] + df_defm_abcd['demandeur_homme_abcd_2549'] + df_defm_abcd['demandeur_homme_abcd_plus50']
df_defm_abcd['demandeur_femme_abcd_total']   = df_defm_abcd['demandeur_femme_abcd_moins25'] + df_defm_abcd['demandeur_femme_abcd_2549'] + df_defm_abcd['demandeur_femme_abcd_plus50']

# Création de la colonne "TOTAL ALL"
df_defm_abcd['demandeur_total_abcd_total']   = df_defm_abcd['demandeur_homme_abcd_total']   + df_defm_abcd['demandeur_femme_abcd_total']

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
        df_defm_abcd[['date_jointure', 'demandeur_total_abcd_total',
                       'demandeur_homme_abcd_total', 'demandeur_femme_abcd_total',
                       'demandeur_total_abcd_moins25', 'demandeur_total_abcd_2549', 'demandeur_total_abcd_plus50',
                       'demandeur_homme_abcd_moins25', 'demandeur_homme_abcd_2549', 'demandeur_homme_abcd_plus50',
                       'demandeur_femme_abcd_moins25', 'demandeur_femme_abcd_2549', 'demandeur_femme_abcd_plus50'
                      ]], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_22484\3969876481.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_defm_abcd['time_period'] = pd.to_datetime(df_defm_abcd['Période'].str.replace('T', '-Q'))


## 2) Offres d'emploi diffusées à France Travail (OEDFT)

In [72]:
df_oedft = dict_dfs['séries_offres_difusees_T32025']

#Renommer les colonnes pour les uniformiser
df_oedft = df_oedft.rename(columns={
    'Nombre d\'offres diffusées': 'nb_offres_france_travail'
})

# Création d'une colonne date aggrégeant les différentes infos de date disponibles
df_oedft['day'] = 1
df_oedft['time_period'] = pd.to_datetime(pd.DataFrame({
    'year': df_oedft['Année'],
    'month': df_oedft['Mois'],
    'day': df_oedft['day']
}))

df_oedft['time_period'] = pd.to_datetime(df_oedft['time_period'], dayfirst=True)
df_oedft['date_jointure'] = df_oedft['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_oedft[['date_jointure', 'nb_offres_france_travail']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 3) Population active (OIT)

In [73]:
## population_active_france-oit-1990_2024
df_pop_active = dict_dfs['population_active_france-oit-1990_2024']

# Renommer les colonnes pour les uniformiser
df_pop_active = df_pop_active.rename(columns={
    'Year':'years',
    'population_active_france': 'population_active'
})

df_pop_active['time_period'] = pd.to_datetime(df_pop_active['years'], format='%Y')
df_pop_active['date_jointure'] = df_pop_active['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_pop_active[['date_jointure', 'population_active']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 4) Taux de chômage en France (INSEE)

In [74]:
# taux_chomage-insee-annuel-1975_2024
df_tx_chomage_insee = dict_dfs['taux_chomage-insee-annuel-1975_2024']

# Renommer les colonnes pour les uniformiser
df_tx_chomage_insee = df_tx_chomage_insee.rename(columns={
    'Année ':'years',
    'Taux de chômage': 'taux_chomage_insee'
})

df_tx_chomage_insee['time_period'] = pd.to_datetime(df_tx_chomage_insee['years'], format='%Y')
df_tx_chomage_insee['date_jointure'] = df_tx_chomage_insee['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_tx_chomage_insee[['date_jointure', 'taux_chomage_insee']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 5) Taux de chômage par pays (OCDE)

In [75]:
# ocde-taux_chomage_mensuel-1984_2025
df_ocde = dict_dfs['taux_chomage-ocde-mensuel-1984_2025']

# Filtrage des données sur la FRANCE
df_ocde = df_ocde[df_ocde['REF_AREA'] == 'FRA']

# Renommer les colonnes pour les uniformiser
df_ocde = df_ocde.rename(columns={
    'OBS_VALUE': 'taux_chomage_ocde'
})

df_ocde['time_period'] = pd.to_datetime(df_ocde['TIME_PERIOD'], dayfirst=True)
df_ocde['date_jointure'] = df_ocde['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ocde[['date_jointure', 'taux_chomage_ocde']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 6) Euribor 3-month (E3M)

In [76]:
# euribor-3_month-1994_2025
df_e3m = dict_dfs['euribor-3_month-1994_2025']

# Renommer les colonnes pour les uniformiser
df_e3m = df_e3m.rename(columns={
    'Euribor 3-month - Historical close, average of observations through period (FM.M.U2.EUR.RT.MM.EURIBOR3MD_.HSTA)': 'taux_euribor_3m'
})

df_e3m['time_period'] = pd.to_datetime(df_e3m['DATE'], dayfirst=True, format='%Y-%m-%d')
df_e3m['date_jointure'] = df_e3m['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_e3m[['date_jointure', 'taux_euribor_3m']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 7) Main refinancing operations (MRO)

In [77]:
# main_refinancing_operations -1999_2025
df_mro = dict_dfs['main_refinancing_operations-1999_2025']

# Renommer les colonnes pour les uniformiser
df_mro = df_mro.rename(columns={
    'Main refinancing operations - fixed rate tenders (fixed rate) (date of changes) - Level (FM.B.U2.EUR.4F.KR.MRR_FR.LEV)': 'MRO'
})

df_mro['time_period'] = pd.to_datetime(df_mro['DATE'], format='%Y-%m-%d')
df_mro['date_jointure'] = df_mro['time_period'].dt.to_period('M').dt.to_timestamp()


# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_mro[['date_jointure', 'MRO']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 8) Indice des prix à la consommation (IPC)

In [78]:
# ipc-glissement_annuel-ensemble_menage-1991_2025
df_ipc = dict_dfs['ipc-glissement_annuel-ensemble_menage-1991_2025']

# Renommer les colonnes pour les uniformiser
df_ipc = df_ipc.rename(columns={
    'obs_value': 'ipc'
})

df_ipc['time_period'] = pd.to_datetime(df_ipc['time_period'], dayfirst=True)
df_ipc['date_jointure'] = df_ipc['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ipc[['date_jointure', 'ipc']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 9) Inflation sous-jacente (ISJ)

In [79]:
# isj-glissement_annuel-ensemble_menage-1991_2025
df_isj = dict_dfs['isj-glissement_annuel-ensemble_menage-1991_2025']

# Renommer les colonnes pour les uniformiser
df_isj = df_isj.rename(columns={
    'obs_value': 'isj'
})

df_isj['time_period'] = pd.to_datetime(df_isj['time_period'], dayfirst=True)
df_isj['date_jointure'] = df_isj['time_period'].dt.to_period('M').dt.to_timestamp()
# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_isj[['date_jointure', 'isj']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 10) Indice des prix à la consommation ENERGIE (IPC ENERGIE)

In [80]:
# ipc_energie_only-glissement_annuel-ensemble_menage-1991_2025
df_ipc_energie = dict_dfs['ipc_energie_only-glissement_annuel-ensemble_menage-1991_2025']

# Renommer les colonnes pour les uniformiser
df_ipc_energie = df_ipc_energie.rename(columns={
    'obs_value': 'ipc_energie_only'
})

df_ipc_energie['time_period'] = pd.to_datetime(df_ipc_energie['time_period'], dayfirst=True)
df_ipc_energie['date_jointure'] = df_ipc_energie['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ipc_energie[['date_jointure', 'ipc_energie_only']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 11) Indicateur du climat des affaires (ICA)

In [81]:
# indicateur_climat_affaires-tous_secteurs-france_métropolitaine-1977_2025
df_ica = dict_dfs['indicateur_climat_affaires-tous_secteurs-france_métropolitaine-1977_2025']

# Renommer les colonnes pour les uniformiser
df_ica = df_ica.rename(columns={
    'obs_value': 'indicateur_climat_affaires'
})

df_ica['time_period'] = pd.to_datetime(df_ica['time_period'], dayfirst=True)
df_ica['date_jointure'] = df_ica['time_period'].dt.to_period('M').dt.to_timestamp()
# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ica[['date_jointure', 'indicateur_climat_affaires']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 12) Indicateur du climat de l'emploi (ICE)

In [82]:
# indicateur_climat_emploi-tous_secteurs-france_métropolitaine-1991_2025
df_ice = dict_dfs['indicateur_climat_emploi-tous_secteurs-france_métropolitaine-1991_2025']

# Renommer les colonnes pour les uniformiser
df_ice = df_ice.rename(columns={
    'obs_value': 'indicateur_climat_emploi'
})

df_ice['time_period'] = pd.to_datetime(df_ice['time_period'], dayfirst=True)
df_ice['date_jointure'] = df_ice['time_period'].dt.to_period('M').dt.to_timestamp()
# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_ice[['date_jointure', 'indicateur_climat_emploi']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 13) Indicateur de retournement conjoncturel (IRC)

In [83]:
# indicateur_retournement_conjoncturel-tous_secteurs-france_métropolitaine-1988_2025
df_irc = dict_dfs['indicateur_retournement_conjoncturel-tous_secteurs-france_métropolitaine-1988_2025']

# Renommer les colonnes pour les uniformiser
df_irc = df_irc.rename(columns={
    'obs_value': 'indicateur_retournement_conjoncturel'
})

df_irc['time_period'] = pd.to_datetime(df_irc['time_period'], dayfirst=True)
df_irc['date_jointure'] = df_irc['time_period'].dt.to_period('M').dt.to_timestamp()
# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_irc[['date_jointure', 'indicateur_retournement_conjoncturel']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 14) Produit intérieur brut total (PIB)

In [84]:
# pib_total-volume_prix_année_précédente_chaînés-série_CVS-CJO-1949_2025
df_pib = dict_dfs['pib_total-volume_prix_année_précédente_chaînés-série_CVS-CJO-1949_2025']

# Renommer les colonnes pour les uniformiser
df_pib = df_pib.rename(columns={
    'obs_value': 'pib'
})

df_pib['time_period'] = pd.to_datetime(df_pib['time_period'].str.replace('-T', 'Q'))

df_pib['date_jointure'] = df_pib['time_period'].dt.to_period('M').dt.to_timestamp()

df_final = pd.merge(
    df_final, 
    df_pib[['date_jointure', 'pib']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_22484\1229989185.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_pib['time_period'] = pd.to_datetime(df_pib['time_period'].str.replace('-T', 'Q'))


## 15) Indice du coût du travail (ICT)

In [85]:
# indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025
df_ict = dict_dfs['indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025']

# Renommer les colonnes pour les uniformiser
df_ict = df_ict.rename(columns={
    'obs_value': 'ict'
})

df_ict['time_period'] = pd.to_datetime(df_ict['time_period'].str.replace('-T', 'Q'))
df_ict['date_jointure'] = df_ict['time_period'].dt.to_period('M').dt.to_timestamp()

df_final = pd.merge(
    df_final, 
    df_ict[['date_jointure', 'ict']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

C:\Users\julie\AppData\Local\Temp\ipykernel_22484\558528430.py:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_ict['time_period'] = pd.to_datetime(df_ict['time_period'].str.replace('-T', 'Q'))


## 16) Défaillances d'entreprises (DE)

In [86]:
# défaillances_entreprises-tous_secteurs-brut-france-1990_2025
df_de = dict_dfs['défaillances_entreprises-tous_secteurs-brut-france-1990_2025']

# Renommer les colonnes pour les uniformiser
df_de = df_de.rename(columns={
    'obs_value': 'nb_defaillances_entreprise'
})

df_de['time_period'] = pd.to_datetime(df_de['time_period'], dayfirst=True)
df_de['date_jointure'] = df_de['time_period'].dt.to_period('M').dt.to_timestamp()

# Fusion des deux dataframes
df_final = pd.merge(
    df_final, 
    df_de[['date_jointure', 'nb_defaillances_entreprise']], 
    left_on='date', 
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## 17) L'emploi intérimaire (EI)

In [87]:
# indice_cout_travail-cout_horaire-ensemble_des_secteurs-base_100_2020-1998_2025
df_ei = dict_dfs['interimaire-evolution-trimestrielle-2000_2025']

# Renommer les colonnes pour les uniformiser
df_ei = df_ei.rename(columns={
    'Niveau': 'nb_interimaires'
})

serie_trimestres = df_ei.iloc[:, 1]
df_ei['time_period'] = pd.PeriodIndex(df_ei.iloc[:, 0].astype(str).str.replace('T', 'Q'), freq='Q').to_timestamp()
df_ei['date_jointure'] = df_ei['time_period'].dt.to_period('M').dt.to_timestamp()

df_final = pd.merge(
    df_final, 
    df_ei[['date_jointure', 'nb_interimaires']], 
    left_on='date',
    right_on='date_jointure', 
    how='left'
)

# On peut supprimer la colonne de jointure en double et trier
df_final = df_final.drop(columns=['date_jointure']).sort_values('date')

## Visualisation du dataframe final

In [88]:
df_final.tail(20)

,date,annee,trimestre,mois,taux_chomage_total_insee,taux_chomage_homme_insee,taux_chomage_femme_insee,taux_chomage_15_24_insee,taux_chomage_25_49_insee,taux_chomage_50_plus_insee,...,ipc,isj,ipc_energie_only,indicateur_climat_affaires,indicateur_climat_emploi,indicateur_retournement_conjoncturel,pib,ict,nb_defaillances_entreprise,nb_interimaires
416,2024-07-01,2024,3,7,7.4,7.6,7.1,19.7,6.6,4.7,...,2.3,1.5,8.5,94.4,97.2,-1.0,655027.0,110.1,5843.0,726325.294868
417,2024-08-01,2024,3,8,NaN,NaN,NaN,NaN,NaN,NaN,...,1.8,1.7,0.4,96.6,98.4,0.7,NaN,NaN,1599.0,NaN
418,2024-09-01,2024,3,9,NaN,NaN,NaN,NaN,NaN,NaN,...,1.1,1.4,-3.3,97.4,98.6,-0.8,NaN,NaN,6314.0,NaN
419,2024-10-01,2024,4,10,7.3,7.4,7.1,18.7,6.6,4.7,...,1.2,1.4,-2.0,97.0,97.4,-0.9,654734.0,111.3,6554.0,713974.330136
420,2024-11-01,2024,4,11,NaN,NaN,NaN,NaN,NaN,NaN,...,1.3,1.5,-0.7,96.1,98.4,-0.7,NaN,NaN,6099.0,NaN
421,2024-12-01,2024,4,12,NaN,NaN,NaN,NaN,NaN,NaN,...,1.3,1.3,1.2,95.0,96.5,-0.9,NaN,NaN,5752.0,NaN
422,2025-01-01,2025,1,1,7.5,7.5,7.4,19.2,6.8,4.8,...,1.7,1.4,2.7,95.4,97.0,-0.7,655389.0,112.3,5948.0,710419.927973
423,2025-02-01,2025,1,2,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8,1.3,-5.8,95.9,93.5,-0.6,NaN,NaN,5826.0,NaN
424,2025-03-01,2025,1,3,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8,1.3,-6.6,96.6,96.4,-0.6,NaN,NaN,6382.0,NaN
425,2025-04-01,2025,2,4,7.6,7.7,7.4,19.1,6.9,4.8,...,0.8,1.3,-7.8,97.1,97.8,-0.3,657523.0,112.2,5863.0,709895.580836


## Export du dataset

In [89]:
repertoire_data_processed = Path.cwd().parent.parent / "data" / "processed"
chemin_export = repertoire_data_processed / "01_dataset_jointure.csv"

df_final.to_csv(chemin_export, index=False, encoding='utf-8')